<a href="https://colab.research.google.com/github/adesam146/nlpcw/blob/sam_preprocessing/NLP_CW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install torchvision torch allennlp

    100% |████████████████████████████████| 5.5MB 7.9MB/s 
    100% |████████████████████████████████| 2.0MB 1.3MB/s 
    100% |████████████████████████████████| 51kB 20.2MB/s 
    100% |████████████████████████████████| 4.5MB 10.9MB/s 
    100% |████████████████████████████████| 512kB 28.5MB/s 
    100% |████████████████████████████████| 1.5MB 19.0MB/s 
    100% |████████████████████████████████| 552kB 25.4MB/s 
    100% |████████████████████████████████| 51kB 22.5MB/s 
    100% |████████████████████████████████| 133kB 30.6MB/s 
    100% |████████████████████████████████| 12.6MB 3.9MB/s 
    100% |████████████████████████████████| 51kB 21.2MB/s 
    100% |████████████████████████████████| 2.4MB 15.1MB/s 
    100% |████████████████████████████████| 143kB 34.2MB/s 
    100% |████████████████████████████████| 61kB 24.1MB/s 
    100% |████████████████████████████████| 51kB 24.2MB/s 
    100% |████████████████████████████████| 102kB 29.2MB/s 
    100% |████████████████████████████████| 204

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import sampler

import torchvision.datasets as dset

import numpy as np
import pandas as pd

import torchvision.transforms as T

from allennlp.modules.elmo import Elmo, batch_to_ids


In [5]:
#Use pretrained ELMO weights. 
options_file = "https://s3-us-west-2.amazonaws.com/allennlp/models/elmo/2x4096_512_2048cnn_2xhighway/elmo_2x4096_512_2048cnn_2xhighway_options.json"
weight_file = "https://s3-us-west-2.amazonaws.com/allennlp/models/elmo/2x4096_512_2048cnn_2xhighway/elmo_2x4096_512_2048cnn_2xhighway_weights.hdf5"

elmo = Elmo(options_file, weight_file, 2, dropout=0)

02/12/2019 12:07:56 - INFO - allennlp.modules.elmo -   Initializing ELMo
02/12/2019 12:07:57 - INFO - allennlp.common.file_utils -   https://s3-us-west-2.amazonaws.com/allennlp/models/elmo/2x4096_512_2048cnn_2xhighway/elmo_2x4096_512_2048cnn_2xhighway_options.json not found in cache, downloading to /tmp/tmp6eyz53h1
100%|██████████| 336/336 [00:00<00:00, 146831.23B/s]
02/12/2019 12:07:57 - INFO - allennlp.common.file_utils -   copying /tmp/tmp6eyz53h1 to cache at /root/.allennlp/cache/1b18dd09179ba1bd92d0f4aa8a4089152c7ec97f14abfbad7052a443c5695047.15c465f454169b1ddb0b7f0d48dca2484f5b3350e81e870a0c3a797233eaac66
02/12/2019 12:07:57 - INFO - allennlp.common.file_utils -   creating metadata file for /root/.allennlp/cache/1b18dd09179ba1bd92d0f4aa8a4089152c7ec97f14abfbad7052a443c5695047.15c465f454169b1ddb0b7f0d48dca2484f5b3350e81e870a0c3a797233eaac66
02/12/2019 12:07:57 - INFO - allennlp.common.file_utils -   removing temp file /tmp/tmp6eyz53h1
02/12/2019 12:07:57 - INFO - allennlp.common.f

In [36]:
#Elmo test
sentences = [['First', 'sentence', '.'], ['Another', '.'], 
             ["Oh", "here", "we", "Go", "now", "you", "fool", "."], 
             ["meaninglesswordnotinvocab"]]
             
character_ids = batch_to_ids(sentences)

# The shape would be (#sentences, length_of_longest_sentence, len of longest word(in original corpus?)=50)
# Note if a word has n characters the first n+2 entries in its size 50 embedding would be used to
# represent it. '+2' because it seems like they implicitly add a start and end of word
# 'character'
print(character_ids[0])

embeddings = elmo(character_ids)

embed = embeddings["elmo_representations"]
print(len(embed))
print(embed[0].shape)
print(embed[1].shape)


tensor([[259,  71, 106, 115, 116, 117, 260, 261, 261, 261, 261, 261, 261, 261,
         261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261,
         261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261,
         261, 261, 261, 261, 261, 261, 261, 261],
        [259, 116, 102, 111, 117, 102, 111, 100, 102, 260, 261, 261, 261, 261,
         261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261,
         261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261,
         261, 261, 261, 261, 261, 261, 261, 261],
        [259,  47, 260, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261,
         261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261,
         261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261,
         261, 261, 261, 261, 261, 261, 261, 261],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,  

In [0]:
# ELMO takes a list of parsed sentences as an input
# It generates an embedding of length 1024 per word
# We then need to find a good method of combining the word vecs to create 
# a sentence embedding (this article is good: https://medium.com/huggingface/universal-word-sentence-embeddings-ce48ddc8fc3a). 


In [0]:
from google.colab import files

#File upload instructions:
  #1. Run this cell
  #2. Press "Choose Files" at the bottom of this cell
  #3. Select offenseval-training-v1.tsv locally to load it as the variable noisy_values  
  
file1 = files.upload()


Saving offenseval-training-v1.tsv to offenseval-training-v1 (1).tsv


In [0]:
train = pd.read_csv("offenseval-training-v1.tsv", delimiter="\t")
print(train.shape)
print(train.head())

(13240, 5)
      id                                              tweet subtask_a  \
0  86426  @USER She should ask a few native Americans wh...       OFF   
1  90194  @USER @USER Go home you’re drunk!!! @USER #MAG...       OFF   
2  16820  Amazon is investigating Chinese employees who ...       NOT   
3  62688  @USER Someone should'veTaken" this piece of sh...       OFF   
4  43605  @USER @USER Obama wanted liberals &amp; illega...       NOT   

  subtask_b subtask_c  
0       UNT       NaN  
1       TIN       IND  
2       NaN       NaN  
3       UNT       NaN  
4       NaN       NaN  
